## POS tagging using HMMs - Lab 9

In [77]:
#All imports
import nltk
from nltk.corpus import brown
from sklearn.model_selection import KFold
import numpy as np
from sklearn.metrics import classification_report
# Download the Brown corpus and the universal tagset
nltk.download("brown")
nltk.download('universal_tagset')

[nltk_data] Downloading package brown to /home/cs240lab/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /home/cs240lab/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [3]:
# Get the tagged sentences from the Brown corpus
tagged_sentences = brown.tagged_sents(tagset="universal")
tagged_sentences = list(tagged_sentences)

In [4]:
tagged_sentences[5]

[('It', 'PRON'),
 ('recommended', 'VERB'),
 ('that', 'ADP'),
 ('Fulton', 'NOUN'),
 ('legislators', 'NOUN'),
 ('act', 'VERB'),
 ('``', '.'),
 ('to', 'PRT'),
 ('have', 'VERB'),
 ('these', 'DET'),
 ('laws', 'NOUN'),
 ('studied', 'VERB'),
 ('and', 'CONJ'),
 ('revised', 'VERB'),
 ('to', 'ADP'),
 ('the', 'DET'),
 ('end', 'NOUN'),
 ('of', 'ADP'),
 ('modernizing', 'VERB'),
 ('and', 'CONJ'),
 ('improving', 'VERB'),
 ('them', 'PRON'),
 ("''", '.'),
 ('.', '.')]

In [5]:
for i in tagged_sentences:
    for j in range(len(i)):
        placeholder=(i[j][0].lower(),i[j][1])
        i[j]=placeholder

tagged_sentences[5]

[('it', 'PRON'),
 ('recommended', 'VERB'),
 ('that', 'ADP'),
 ('fulton', 'NOUN'),
 ('legislators', 'NOUN'),
 ('act', 'VERB'),
 ('``', '.'),
 ('to', 'PRT'),
 ('have', 'VERB'),
 ('these', 'DET'),
 ('laws', 'NOUN'),
 ('studied', 'VERB'),
 ('and', 'CONJ'),
 ('revised', 'VERB'),
 ('to', 'ADP'),
 ('the', 'DET'),
 ('end', 'NOUN'),
 ('of', 'ADP'),
 ('modernizing', 'VERB'),
 ('and', 'CONJ'),
 ('improving', 'VERB'),
 ('them', 'PRON'),
 ("''", '.'),
 ('.', '.')]

Creating Mapping from States and Observation Traces to numbers

In [6]:
states=[]
observations=[]

for j in tagged_sentences:
    for i in j:
        if not(i[-1] in states):
            states.append(i[-1])
        if not(i[0] in observations):
            observations.append(i[0])

print(len(states),len(observations))

12 49815


In [7]:
#mapping from states to numbers (indices in matrix)
stateMap={'BOS':0}
j=1
for i in states:
    stateMap[i]=j
    j=j+1
stateMap['EOS']=j
# print(stateMap)

numToStates={}
j=0
for i in stateMap:
    numToStates[str(j)]=i
    j=j+1

print(numToStates)

#mapping from observation traces to numbers
obsMap={'eps':0}
k=1
for i in observations:
    obsMap[i]=k
    k=k+1
        

{'0': 'BOS', '1': 'DET', '2': 'NOUN', '3': 'ADJ', '4': 'VERB', '5': 'ADP', '6': '.', '7': 'ADV', '8': 'CONJ', '9': 'PRT', '10': 'PRON', '11': 'NUM', '12': 'X', '13': 'EOS'}


In [8]:
#using KFold from sklearn to create folds
kf=KFold(n_splits=5,shuffle=False)

OneSmoothedTrans=[]
OneSmoothedEmm=[]

for trainIndices,testIndices in kf.split(tagged_sentences):

    training=[tagged_sentences[i] for i in trainIndices]
    test=[tagged_sentences[i] for i in testIndices]

    obsStateMatrix=np.zeros((len(states)+2,len(observations)+1))+1
    stateMatrix=np.zeros((len(states)+2,len(states)+2))+1
    y=len(tagged_sentences)
    for i in range(y):
        z=len(tagged_sentences[i])
        for j in range(z):
            if j==0:
                stateMatrix[stateMap['BOS']][stateMap[tagged_sentences[i][j][-1]]]=stateMatrix[stateMap['BOS']][stateMap[tagged_sentences[i][j][-1]]]+1
                stateMatrix[stateMap['BOS']][obsMap['eps']]=stateMatrix[stateMap['BOS']][obsMap['eps']]+1
            if j==z-1:
                stateMatrix[stateMap['EOS']][stateMap['BOS']]=stateMatrix[stateMap['EOS']][stateMap['BOS']]+1
                stateMatrix[stateMap['EOS']][obsMap['eps']]=stateMatrix[stateMap['EOS']][obsMap['eps']]+1
            if j<z-1:
                nextState=tagged_sentences[i][j+1][-1]
                currObs=tagged_sentences[i][j][0]
                currState=tagged_sentences[i][j][-1]

                stateMatrix[stateMap[currState]][stateMap[nextState]]=stateMatrix[stateMap[currState]][stateMap[nextState]]+1
                obsStateMatrix[stateMap[currState]][obsMap[currObs]]=obsStateMatrix[stateMap[currState]][obsMap[currObs]]+1
                
    stateMatrix=stateMatrix/(stateMatrix.sum(axis=1,keepdims=True))
    obsStateMatrix=obsStateMatrix/(obsStateMatrix.sum(axis=1,keepdims=True))
    OneSmoothedEmm.append(obsStateMatrix)
    OneSmoothedTrans.append(stateMatrix)
                

<h3>Split 1</h3>

In [9]:
# print("transition matrix")
# print(OneSmoothedTrans[0])
print()
print("Emission Matrix")
print(OneSmoothedEmm[0].shape)


Emission Matrix
(14, 49816)


In [75]:
# def viterbi(obs, states, start_p, trans_p, emit_p):
#     V = [{}]
#     path = {}
#     for y in states:
#         V[0][y] = start_p[y] * emit_p[y][obs[0]]
#         path[y] = [y]
#     for t in range(1, len(obs)):
#         V.append({})
#         newpath = {}
#         for y in states:
#             (prob, state) = max([(V[t-1][y0] * trans_p[y0][y] * emit_p[y][obs[t]], y0) for y0 in states])
#             V[t][y] = prob
#             newpath[y] = path[state] + [y]
#         path = newpath
#     (prob, state) = max([(V[-1][y], y) for y in states])
#     return (prob, path[state])


def viterbiCS217(observe:str, trans_p:np.array, emit_p:np.array):
    # return
    observe.insert(0,'eps')
    N=len(stateMap)
    t=len(observe)
    seqScore=np.zeros((N,t))
    seqScore[0][obsMap['eps']]=1 #start with BOS and epsilon char
    for i in range(1,N):
        seqScore[i][obsMap['eps']]=0
    backPtr=np.zeros((N,t))
    backPtr.astype(int)
    backPtr[0][0]=None
    # count=0

    for t_iter in range(1,t):
        for i in range(1,N):
            seqScore[i][t_iter],backPtr[i][t_iter]=max([(seqScore[j][t_iter-1] * trans_p[j][i] * emit_p[i][obsMap[observe[t_iter]]], j) for j in range(0,N)])
            
    s,currLast=max([(seqScore[j][t-1],j) for j in range(0,N)])
    position=t
    path=[]
    while position!=0:
        path.append(currLast)
        currLast=backPtr[int(currLast)][position-1]
        position=position-1

    path.reverse()
    path.append(stateMap['EOS'])

    return path

def answer(sentence,transiton,emmission):
    path=viterbiCS217(sentence,transiton,emmission)
    for i in range(len(path)):
         path[i]=int(path[i])
    # path=path.astype(int)
    posTagging=[]
    for i in range(1,len(sentence)):
        posTagging.append(numToStates[str(int(path[i]))])
    
    return posTagging


sentence=[]
for i in tagged_sentences[5]:
        sentence.append(i[0])
print(answer(sentence,OneSmoothedTrans[0],OneSmoothedEmm[0]))

['PRON', 'VERB', 'ADP', 'NOUN', 'NOUN', 'NOUN', '.', 'PRT', 'VERB', 'DET', 'NOUN', 'VERB', 'CONJ', 'VERB', 'ADP', 'DET', 'NOUN', 'ADP', 'NOUN', 'CONJ', 'VERB', 'PRON', '.', 'CONJ']


In [54]:
obsStateMatrix=np.zeros((len(states)+2,len(observations)+1))+1
stateMatrix=np.zeros((len(states)+2,len(states)+2))+1
y=len(tagged_sentences)
for i in range(y):
    z=len(tagged_sentences[i])
    for j in range(z):
        if j==0:
            stateMatrix[stateMap['BOS']][stateMap[tagged_sentences[i][j][-1]]]=stateMatrix[stateMap['BOS']][stateMap[tagged_sentences[i][j][-1]]]+1
            stateMatrix[stateMap['BOS']][obsMap['eps']]=stateMatrix[stateMap['BOS']][obsMap['eps']]+1
        if j==z-1:
            stateMatrix[stateMap['EOS']][stateMap['BOS']]=stateMatrix[stateMap['EOS']][stateMap['BOS']]+1
            stateMatrix[stateMap['EOS']][obsMap['eps']]=stateMatrix[stateMap['EOS']][obsMap['eps']]+1
        if j<z-1:
            nextState=tagged_sentences[i][j+1][-1]
            currObs=tagged_sentences[i][j][0]
            currState=tagged_sentences[i][j][-1]
            stateMatrix[stateMap[currState]][stateMap[nextState]]=stateMatrix[stateMap[currState]][stateMap[nextState]]+1
            obsStateMatrix[stateMap[currState]][obsMap[currObs]]=obsStateMatrix[stateMap[currState]][obsMap[currObs]]+1                
stateMatrix=stateMatrix/(stateMatrix.sum(axis=1,keepdims=True))
obsStateMatrix=obsStateMatrix/(obsStateMatrix.sum(axis=1,keepdims=True))

In [79]:
kf=KFold(n_splits=5,shuffle=False)

step=0
answers=[]
labels=[]

for trainIndices,testIndices in kf.split(tagged_sentences):
    print(step)
    training=[tagged_sentences[i] for i in trainIndices]
    test=[tagged_sentences[i] for i in testIndices]
    # print(testIndices)
    sentences=[]
    for i in range(11468):
        # print(testIndices[i],end="")
        if i%1000==0:
             print(i)
        testSentence=[]
        for j in test[i]:
                testSentence.append(j[0])
                labels.append(j[1])
        sentences.append(testSentence)
        
    #testing
    for sent in sentences:
        result=answer(sent,OneSmoothedTrans[step],OneSmoothedEmm[step])
        answers=answers+result

    step=step+1

0
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
1
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
2
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
3
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
4
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000


In [80]:
classification_report(labels,answers)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


'              precision    recall  f1-score   support\n\n           .       0.96      0.88      0.92    147565\n         ADJ       0.88      0.91      0.90     83721\n         ADP       0.92      0.97      0.95    144766\n         ADV       0.83      0.88      0.85     56239\n        CONJ       0.90      0.99      0.94     38151\n         DET       0.92      0.99      0.95    137019\n         EOS       0.00      0.00      0.00         0\n        NOUN       0.95      0.92      0.94    275558\n         NUM       0.98      0.83      0.90     14874\n        PRON       0.92      0.96      0.94     49334\n         PRT       0.91      0.86      0.88     29829\n        VERB       0.96      0.93      0.94    182750\n           X       0.72      0.44      0.54      1386\n\n    accuracy                           0.93   1161192\n   macro avg       0.83      0.81      0.82   1161192\nweighted avg       0.93      0.93      0.93   1161192\n'

In [ ]:
        precision    recall  f1-score   support\n\n           
.       0.96      0.88      0.92    147565\n         
ADJ       0.88      0.91      0.90     83721\n         
ADP       0.92      0.97      0.95    144766\n         
ADV       0.83      0.88      0.85     56239\n        
CONJ       0.90      0.99      0.94     38151\n         
DET       0.92      0.99      0.95    137019\n         
EOS       0.00      0.00      0.00         0\n        
NOUN       0.95      0.92      0.94    275558\n         
NUM       0.98      0.83      0.90     14874\n        
PRON       0.92      0.96      0.94     49334\n         
PRT       0.91      0.86      0.88     29829\n        
VERB       0.96      0.93      0.94    182750\n           
X       0.72      0.44      0.54      1386\n\n    
accuracy                           0.93   1161192\n   
macro avg       0.83      0.81      0.82   1161192\n
weighted avg       0.93      0.93      0.93   1161192\n